# Redis Basics

In [7]:
# import redis
import redis
import os

In [8]:
# connect redis to localhost and port
redis = redis.Redis(host='localhost', port=6379, db=0)

In [17]:
# simple commands to test
redis.set('foo','bar')
redis.get('foo')

b'bar'

In [18]:
for key in redis.scan_iter("*"):
    print(key)

b'foo'


In [16]:
#redis.flushdb()

True

## Lua Scripts

In [19]:
stats_script = """
    -- Convert arguments to numbers
    local k1 = redis.call('get', KEYS[1])
    local k2 = redis.call('get', KEYS[2])

    if ARGV[1] == "sum" then
      return k1 + k2
    elseif ARGV[1] == "max" then
      return math.max(k1, k2)
    else
      return nil
    end
"""

In [20]:
redis.set("hits:homepage", 2000)
redis.set("hits:loginpage", 75)

# Register our script with the Redis Python client and
# return a callable object for invoking our script.
stats = redis.register_script(stats_script)

# Invoke our "sum" script.
# This calls SCRIPT LOAD and then stores
# the SHA1 digest of the script for future use.
total = stats(["hits:homepage", "hits:loginpage"], ["sum"])
print(total)
assert(total == 2075)

# Two more tests.
max = stats(["hits:homepage", "hits:loginpage"], ["max"])
print(max)
assert(max == 2000)

2075
2000


In [21]:
redis.get("hits:homepage")

b'2000'

## Lua Script (ZINCRBY and ZINCRBYFLOAT)

In [33]:
zscript_lua ="""
    -- explain
    local zsetMembers = redis.call('zrange', KEYS[1], '0', '-1')
    for k,member in pairs(zsetMembers) do
        redis.call('zincrby', KEYS[1], 1, member)
    end
"""

In [39]:
redis.zadd("myzset",{'one':1})
redis.zadd("myzset", {'two':2})

print(redis.zrange('myzset', 0,-1,withscores = True))

# Register our script with the Redis Python client and
# return a callable object for invoking our script.
lua_script = redis.register_script(zscript_lua)


[(b'one', 1.0), (b'two', 2.0)]


In [41]:
# Invoke our "sum" script.
# This calls SCRIPT LOAD and then stores
# the SHA1 digest of the script for future use.
total = lua_script(["myzset"])
print(total)

None


In [42]:
print(redis.zrange('myzset', 0,-1,withscores = True))

[(b'one', 2.0), (b'two', 3.0)]


In [43]:
########################################

In [62]:
zscript_lua ="""
    -- first set a local var, zsetMembers, this will call the whole zset range
    local zsetMembers = redis.call('zrange', KEYS[1], '0', '-1')
    -- set k2 to be equal to the a KV, and grab the value, the value will be what you incrby
    local k2 = redis.call('get', KEYS[2])
    
    -- loop through each zset
    for k,member in pairs(zsetMembers) do
        
        -- for each zset incrby the k2 val
        redis.call('zincrby', KEYS[1], k2, member)
    end
"""


redis.zadd("myzset",{'one':1})
redis.zadd("myzset", {'two':2})
redis.zadd("myzset", {'three':3})
redis.set("myincrby", 75.5)

print('original')
print(redis.zrange('myzset', 0,-1,withscores = True))

# Register our script with the Redis Python client and
# return a callable object for invoking our script.
lua_script = redis.register_script(zscript_lua)

# Invoke our "sum" script.
# This calls SCRIPT LOAD and then stores
# the SHA1 digest of the script for future use.
total = lua_script(["myzset","myincrby"])
print('After lua')
print(redis.zrange('myzset', 0,-1,withscores = True))

original
[(b'one', 1.0), (b'two', 2.0), (b'three', 3.0)]
After lua
[(b'one', 76.5), (b'two', 77.5), (b'three', 78.5)]
